# EEP 153 Project 2 Subsistence Cost Diet: Atwater

In [10]:
%pip install -r requirements.txt

  Using cached Pint-0.20.1-py3-none-any.whl (269 kB)
  Using cached python_gnupg-0.5.0-py2.py3-none-any.whl (18 kB)
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
  Using cached fooddatacentral-1.0.9-py3-none-any.whl
  Attempting uninstall: pint
    Found existing installation: Pint 0.17
    Uninstalling Pint-0.17:
      Successfully uninstalled Pint-0.17
Note: you may need to restart the kernel to use updated packages.


In [27]:
# API key for Gov;
apikey = "bwFohFv0W79JagEjhjfy121CHf29UEljz0OYel1N"

## Dietary Reference Intakes

In [28]:
# read in dietary requirements (max and min)
import pandas as pd
dri_max = pd.read_csv("Dietary Requirements Max.csv").set_index('Nutrition')

dri_min = pd.read_csv("Dietary Requirements Min.csv").set_index('Nutrition')

# convert units from kcal to kJ
temp = dri_max.loc['Energy']
temp.iloc[1:] = temp.iloc[1:] * 4.184
dri_max.loc['Energy'] = temp

temp = dri_min.loc['Energy']
temp.iloc[1:] = temp.iloc[1:] * 4.184
dri_min.loc['Energy'] = temp

In [29]:
import re
# function for age-sex specific DRI
def dri(age,sex,dietmin,dietmax):
    if age <= 3:
        index = 2

    else:
        for i, j in enumerate(list(dietmax)[3:]):
            if j[0] == sex:
                interval = re.findall(r'\d+', j)
                if len(interval) == 1:
                    if age >= int(interval[0]):
                        index = i + 3
                        break
                else:
                    if age >= int(interval[0]) and age <= int(interval[1]):
                        index = i + 3
                        break

    df = pd.DataFrame({'Nutrition': dietmin.iloc[:, 0],
                       'Max/Min': 'Minimum',
                       'Age & Sex': list(dietmin)[index],
                      'Intake': dietmin.iloc[:, index]})   
    df_max = pd.DataFrame({'Nutrition': dietmax.iloc[:, 0],
                       'Max/Min': 'Maximum',
                       'Age & Sex': list(dietmax)[index],
                      'Intake': dietmax.iloc[:, index]})   

    df = pd.concat([df, df_max], axis=0)
    
    return df
    

In [6]:
# Testing: dietary reference intakes for 20-year-old female
dri(20, 'F', dri_min, dri_max)

,Nutrition,Max/Min,Age & Sex,Intake
Nutrition,,,,
Energy,---,Minimum,F 19-30,8368.0
Protein,RDA,Minimum,F 19-30,46.0
"Fiber, total dietary",---,Minimum,F 19-30,28.0
"Folate, DFE",RDA,Minimum,F 19-30,400.0
"Calcium, Ca",RDA,Minimum,F 19-30,1000.0
"Carbohydrate, by difference",RDA,Minimum,F 19-30,130.0
"Iron, Fe",RDA,Minimum,F 19-30,18.0
"Magnesium, Mg",RDA,Minimum,F 19-30,310.0
Niacin,RDA,Minimum,F 19-30,14.0


## Data on prices for different foods

In [30]:
# get food list from four restaurants
food_list_total = pd.read_csv("FoodList.csv")
food_list_total = food_list_total.astype({"FDC": str})
food_list_total

,Restaurant,Dish,Ingredients,Dish_Price,Quantity,Unit,FDC,Calorie/100g,Ingredient_Price/100 gm
0,Thai Basil,Pineapple Fried Rice,prawns,17.45,140.0000,gram,175180,99.0,1.200
1,Thai Basil,Pineapple Fried Rice,chicken,17.45,200.0000,gram,331960,152.5,0.490
2,Thai Basil,Pineapple Fried Rice,egg,17.45,48.0000,gram,748967,145.0,0.500
3,Thai Basil,Pineapple Fried Rice,Pineapple,17.45,180.0000,gram,2346398,57.0,0.520
4,Thai Basil,Pineapple Fried Rice,white onion,17.45,80.0000,gram,1104962,34.5,0.498
...,...,...,...,...,...,...,...,...,...
181,Poke Bar,Wazzup Poke Bowl,cucumber,15.95,28.3500,gram,168409,15.0,0.880
182,Poke Bar,Wazzup Poke Bowl,green onion,15.95,7.0875,gram,170006,27.0,0.440
183,Poke Bar,Wazzup Poke Bowl,ponzu,15.95,35.4375,gram,2451144,33.0,1.900
184,Poke Bar,Wazzup Poke Bowl,wasabi,15.95,452.1825,gram,171831,292.0,8.780


In [8]:
# construct food lists by restaurant
grouped = food_list_total.groupby(food_list_total.Restaurant)
restaurants = ['Thai Basil', 'Ttoust', 'IB', 'Poke Bar']

food_list_res = []

for r in restaurants: 
    food_list_res.append(grouped.get_group(r))
    

In [36]:
import fooddatacentral as fdc
import warnings
from collections import defaultdict

# get nutritional information for ingredients
ing_res = []
ing_res_list = []
for i in range(4):
    L = []
    D = {}
    items = []
    count = 0
    food_list = food_list_res[i]
    for food in food_list.Ingredients.tolist():
        try:
            FDC = food_list.iloc[count,:].FDC
            count+=1
            temp = fdc.nutrients(apikey,FDC)
            key = temp.Units
            # convert units if necessary
            if 'Energy' in key.index:
                if key['Energy'] != 'kJ':
                    temp.Quantity['Energy'] = temp.Quantity['Energy']*4.184

            L.append(temp.Quantity)
            D[food] = temp.Quantity
            items.append(food)
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))     

    # construct nutrient tables
    # list version
    FoodNutrients_Ing = pd.DataFrame(L,dtype=float)
    FoodNutrients_Ing.index = items
    FoodNutrients_Ing = FoodNutrients_Ing.fillna(0)
    FoodNutrients_Ing = FoodNutrients_Ing.transpose() 
    ing_res_list.append(FoodNutrients_Ing)
    # dictionary version
    FoodNutrients_Ing_d = pd.DataFrame(D,dtype=float)
    FoodNutrients_Ing_d = FoodNutrients_Ing_d.fillna(0)
    ing_res.append(FoodNutrients_Ing_d)

In [37]:
# example: nutrient table for Thai Basil
ing_res_list[0]

,prawns,chicken,egg,Pineapple,white onion,raisins,cashew nuts,peas,carrots,chicken,...,pork,imitation crab,prawns,mushrooms,white onion,rice,catfish,eggplant,red curry,rice
Proximates,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.0,...,0.00,0.00000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
Water,74.33,65.30,75.80,84.990000,91.30,15.500000,5.20,78.86,87.72000,93.9,...,24.76,74.70000,74.33,88.600000,91.30,11.60,79.06,92.30,0.000000,11.60
Energy,415.00,695.00,617.00,0.000000,148.00,5234.250944,2314.00,339.00,0.00000,84.0,...,2645.00,1663.05632,415.00,0.000000,148.00,1500.00,496.00,104.00,2328.278848,1500.00
Protein,23.98,32.10,12.40,0.460938,0.89,3.300000,18.22,5.42,0.94125,0.7,...,9.25,7.62000,23.98,2.414375,0.89,6.94,15.23,0.98,6.670000,6.94
Total lipid (fat),0.28,3.24,9.96,0.211300,0.13,0.250000,43.85,0.40,0.35060,0.1,...,65.70,0.46000,0.28,0.195000,0.13,1.30,5.94,0.18,3.330000,1.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Beta-sitostanol,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.0,...,0.00,0.00000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
Delta-5-avenasterol,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.0,...,0.00,0.00000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
Delta-7-Stigmastenol,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.0,...,0.00,0.00000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
Ergothioneine,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.0,...,0.00,0.00000,0.00,11.060000,0.00,0.00,0.00,0.00,0.000000,0.00


In [46]:
# construct price vector for dishes
# Convert food quantities to FDC units
food_list_total['FDC Quantity'] = food_list_total[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

food_list_total['FDC Price'] = food_list_total['Dish_Price']/food_list_total.groupby('Dish',sort=False)['FDC Quantity'].sum()

food_list_total.dropna(how='any') 

Dish_Prices = food_list_total.groupby('Dish',sort=False)['Dish_Price'].min()/food_list_total.groupby('Dish',sort=False)['FDC Quantity'].sum()

## Table of nutritional information for dishes

In [60]:
# construct nutrient table for dishes
FoodNutrients_Ing_total = pd.DataFrame()
for t in ing_res_list:
    FoodNutrients_Ing_total = pd.concat([FoodNutrients_Ing_total, t], axis=1)
    
FoodNutrients_Ing_total = FoodNutrients_Ing_total.fillna(0)


dishes = food_list_total.groupby('Dish',sort=False)['Ingredients'].count()

FoodNutrients = pd.DataFrame(columns=dishes.keys().tolist(),
                  index=FoodNutrients_Ing.index, )

# sum up nutrients of each ingredient of the dishes
start = 0
for i, column in enumerate(FoodNutrients): 

    FoodNutrients[column] = FoodNutrients_Ing_total.iloc[:, start:start+dishes[i]].sum(axis=1)
    start = start+dishes[i]

FoodNutrients

,Pineapple Fried Rice,Basil Eggplant Chicken,Pad Kee Mow Chicken,Pad Thai Chicken & Prawns,Combo Garlic Pepper,Spicy Basil Catfish,Bibim Bop,Kimchi Soup,Galbi Short Rib,Spicy Rice Cake,...,Veggie Combo,IB's Original(chicken),IB's Original(beef),IB's Original(lamb),Original Salmon Poke Bowl,Firecracker Poke Bowl,Sunset House Poke Bowl,The O.G. Poke Bowl,Goodie Mob Poke Bowl,Wazzup Poke Bowl
Proximates,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
Water,579.000000,473.040000,513.500000,474.33,504.290000,182.960000,5.141800e+02,4.503900e+02,4.079100e+02,5.425200e+02,...,5.997200e+02,4.979900e+02,5.063900e+02,4.921600e+02,349.970000,3.493700e+02,4.245400e+02,3.368300e+02,444.350000,448.360000
Energy,9762.250944,1930.000000,4395.477952,4972.00,7578.056320,4428.278848,3.699143e+08,3.727082e+08,3.751580e+08,3.723758e+08,...,1.561929e+08,1.561928e+08,1.561926e+08,1.561933e+08,203829.476256,3.047023e+06,2.862573e+06,2.914639e+06,62263.178881,15002.014812
Protein,97.712187,13.541875,25.511875,102.38,106.594375,29.820000,4.609000e+01,9.712000e+01,1.153900e+02,3.242000e+01,...,5.016000e+01,7.982000e+01,7.112000e+01,6.428000e+01,42.170000,3.061000e+01,5.624000e+01,4.455000e+01,41.900000,33.200000
Total lipid (fat),58.671900,2.505600,11.285600,69.83,73.785000,10.750000,1.404000e+01,1.132000e+02,9.508000e+01,5.040000e+00,...,1.456100e+02,1.330100e+02,1.322500e+02,1.531800e+02,81.880000,1.200000e+02,7.125000e+01,6.389000e+01,13.850000,46.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Fatty acids, total trans-polyenoic",0.000000,0.000000,0.000000,0.00,0.132000,0.024000,0.000000e+00,1.320000e-01,0.000000e+00,0.000000e+00,...,3.340000e-01,3.340000e-01,3.340000e-01,3.340000e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.023000
"Fluoride, F",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,4.100000e+01,1.100000e+00,1.100000e+00,6.400000e+00,...,2.300000e+00,2.300000e+00,2.300000e+00,2.300000e+00,0.000000,1.300000e+00,1.300000e+00,0.000000e+00,1.300000,1.300000
Phytosterols,0.000000,7.000000,0.000000,0.00,0.000000,7.000000,0.000000e+00,1.500000e+01,7.670000e+02,4.100000e+01,...,4.500000e+01,4.500000e+01,4.500000e+01,4.500000e+01,0.000000,1.400000e+01,1.400000e+01,0.000000e+00,14.000000,14.000000
"Sugars, added",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+02,1.000000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.330000e+01,1.330000e+01,1.330000e+01,0.000000,0.000000


In [55]:
# price vector for dishes
Dish_Prices

Dish
Pineapple Fried Rice          2.326666666666666 / hectogram
Basil Eggplant Chicken       1.9074074074074074 / hectogram
Pad Kee Mow Chicken           2.653225806451613 / hectogram
Pad Thai Chicken & Prawns     1.852497096399535 / hectogram
Combo Garlic Pepper          1.8911335578002246 / hectogram
Spicy Basil Catfish          1.8994708994708995 / hectogram
Bibim Bop                     3.021598272138229 / hectogram
Kimchi Soup                  2.7539370078740157 / hectogram
Galbi Short Rib               4.401988636363637 / hectogram
Spicy Rice Cake               1.635846372688478 / hectogram
BBQ Chicken                   2.013888888888889 / hectogram
Cheeseburger                  2.472527472527472 / hectogram
Avogobble sandwich           3.1055900621118013 / hectogram
Veggie Combo                  3.535353535353536 / hectogram
IB's Original(chicken)       2.2569444444444446 / hectogram
IB's Original(beef)          2.2569444444444446 / hectogram
IB's Original(lamb)          2.2569

In [48]:
# construct price vector for ingredients
ing_prices = []
for r in food_list_res:
    food_list = r
    food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    food_list['FDC Price'] = food_list['Ingredient_Price/100 gm']

    food_list.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    Ingredient_Prices = food_list.groupby('Ingredients',sort=False)['FDC Price'].min()
    ing_prices.append(Ingredient_Prices)

In [49]:
# Example: ingredient prices for Poke bar (per 100g)
ing_prices[3]

Ingredients
salmon            2.20
white rice        0.28
sweet onion       0.44
green onion       0.44
kale              0.88
shoyu             1.69
spicy mayo        1.38
ahi tuna          2.36
cucumber          0.88
sweet chili       1.05
house dressing    3.17
edamame           0.56
ponzu             1.90
tofu              0.74
wasabi            8.78
mayo              0.56
Name: FDC Price, dtype: float64

## Solution

In [50]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)
    #print(Aall)
    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    #print(dietmin)
    #print(Amin)
    Amin = Amin.reindex(dietmin.index,axis=0)
    #print(dietmin.index)
    #print(Amin)
    idx = Amin.index.to_frame()
    #print(Amin)
    idx['type'] = 'min'
    #print(Amin)
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    

    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
    #print(p)
    #print(A)
    #print(b)
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point', options={'presolve': True,
                                                            'cholesky':False, 
                                                            'sym_pos':False,
                                                            'lstsq':True})

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

### Dish-based Solution

In [66]:
#dish-based result
group = 'F 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Dish_Prices,dri_min[group],dri_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

# no feasible solution

Cost of diet for F 19-30 is $5.74 per day.


Diet (in 100s of grams or milliliters):
Series([], dtype: float64)


With the following nutritional outcomes of interest:
                                Outcome  Recommendation
Nutrition                                              
Energy                              NaN          8368.0
Protein                             NaN            46.0
Fiber, total dietary                NaN            28.0
Folate, DFE                         NaN           400.0
Calcium, Ca                         NaN          1000.0
Carbohydrate, by difference         NaN           130.0
Iron, Fe                            NaN            18.0
Magnesium, Mg                       NaN           310.0
Niacin                              NaN            14.0
Phosphorus, P                       NaN           700.0
Potassium, K                        NaN          4700.0
Riboflavin                          NaN             1.1
Thiamin                             NaN          

/tmp/ipykernel_30/2046845037.py:91: UserWarning:

The solution does not satisfy the constraints within the required tolerance of 3.16E-04, yet no errors were raised and there is no certificate of infeasibility or unboundedness. Check whether the slack and constraint residuals are acceptable; if not, consider enabling presolve, adjusting the tolerance option(s), and/or using a different method. Please consider submitting a bug report.



### Ingredient-based solutions

In [79]:
# ingredient-based results for each of 4 restaurants
groups = dri_min.columns[1:]

cost_tbl = pd.DataFrame(columns = groups, index = restaurants)
diet_tbl_name = pd.DataFrame(columns = groups, index = restaurants)
diet_tbl = pd.DataFrame(columns = groups, index = restaurants)
nurt_tbl = pd.DataFrame(columns = groups, index = restaurants)
excess_tbl = pd.DataFrame(columns = groups, index = restaurants)

for group in groups:
    tol = 1e-6
    for i in range(4):
        result = solve_subsistence_problem(ing_res[i],ing_prices[i],dri_min[group],dri_max[group],tol=tol)
        
        #results_tbl.loc[restaurants[i], group] = result
        
        print("Result for restaurant: " + restaurants[i])
        print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))
        
        cost_tbl.loc[restaurants[i], group] = result.fun
        
        # Put back into nice series
        diet = result.diet

        print("\nDiet (in 100s of grams or milliliters):")
        print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
        print()
        
        diet_tbl_name.loc[restaurants[i], group] = list(diet[diet >= tol].index)
        diet_tbl.loc[restaurants[i], group] = list(diet[diet >= tol])

        tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
        print("\nWith the following nutritional outcomes of interest:")
        print(tab)
        print()
        
        nurt_tbl.loc[restaurants[i], group] = list(np.abs(result.A).dot(diet))

        print("\nConstraining nutrients are:")
        excess = tab.diff(axis=1).iloc[:,1]
        print(excess.loc[np.abs(excess) < tol*100].index.tolist())
        
        excess_tbl.loc[restaurants[i], group] = list(excess.loc[np.abs(excess) < tol*100].index.tolist())

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



Result for restaurant: Thai Basil
Cost of diet for C 1-3 is $4.05 per day.


Diet (in 100s of grams or milliliters):
egg         1.653595
carrots     1.385275
rice        0.709294
broccoli    0.294118
peanuts     3.263896
tofu        1.812969
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          10460.000000          4184.0
Protein                           129.501998            13.0
Fiber, total dietary               34.722740            14.0
Folate, DFE                       780.071041           150.0
Calcium, Ca                       700.000000           700.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           14.715686             7.0
Magnesium, Mg                     616.643092            80.0
Niacin                             43.412963             6.0
Phosphorus, P   

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



Result for restaurant: Ttoust
Cost of diet for F 4-8 is $4.52 per day.


Diet (in 100s of grams or milliliters):
Egg            1.176470
Spinach        2.551018
Sesame         0.417671
Brown Sugar    0.000224
Flour          5.737211
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          10430.963777          5020.8
Protein                            84.704676            19.0
Fiber, total dietary               86.208429            16.8
Folate, DFE                       573.604638           200.0
Calcium, Ca                      1000.000014          1000.0
Carbohydrate, by difference       447.820007           130.0
Iron, Fe                           38.434008            10.0
Magnesium, Mg                    1053.206589           130.0
Niacin                             33.812655             8.0
Phosphorus, P             

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



...

Result for restaurant: Thai Basil
Cost of diet for M 31-50 is $6.69 per day.


Diet (in 100s of grams or milliliters):
egg         4.947712
carrots     3.846583
rice        0.179986
broccoli    1.176471
peanuts     3.770187
tofu        1.708200
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12970.399906          9204.8
Protein                           182.869416            56.0
Fiber, total dietary               48.402665            30.8
Folate, DFE                       901.074694           400.0
Calcium, Ca                      1000.000001          1000.0
Carbohydrate, by difference       130.000001           130.0
Iron, Fe                           22.193402             8.0
Magnesium, Mg                     777.823418           420.0
Niacin                             53.041107            16.0
Phosphorus, P 

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



Result for restaurant: Ttoust
Cost of diet for F 51+ is $5.83 per day.


Diet (in 100s of grams or milliliters):
Egg                2.352941
Purple Cabbage     0.042080
Spinach            7.408311
Jasmine Rice       0.000003
Flour              2.352572
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          6695.011473          6694.4
Protein                           73.847559            46.0
Fiber, total dietary              48.687013            22.4
Folate, DFE                     1148.242873           400.0
Calcium, Ca                     1200.000000          1200.0
Carbohydrate, by difference      205.569852           130.0
Iron, Fe                          39.138789             8.0
Magnesium, Mg                    947.270804           320.0
Niacin                            16.226865            14.0
Phosphorus, P    

Result for restaurant: IB
Cost of diet for M 51+ is $7.20 per day.


Diet (in 100s of grams or milliliters):
bun                0.416821
ketchup            0.221608
lettuce            1.015800
cheddar cheese     0.757351
sandwich bun       0.416821
avocado           11.906505
cheese             0.757351
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12370.143726          8368.0
Protein                            75.227445            56.0
Fiber, total dietary               83.184901            28.0
Folate, DFE                      1192.022976           400.0
Calcium, Ca                      1396.454851          1000.0
Carbohydrate, by difference       130.000001           130.0
Iron, Fe                           10.424396             8.0
Magnesium, Mg                     420.000002           420.0
Niacin         

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



In [80]:
# table of minimum diets costs by age-sex groups and restaurants
cost_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,4.052806,5.681348,5.681346,7.366037,7.347199,7.559411,7.889343,6.257664,6.693507,6.257664,6.693507,7.163111,6.693506
Ttoust,3.254792,4.524625,4.524625,5.881581,5.881581,6.141801,6.161338,5.353084,5.414411,5.353083,5.414411,5.831713,5.414411
IB,5.04685,6.000173,6.000173,6.070028,6.050849,9.07069,7.207749,10.594395,7.080006,10.594395,7.198853,6.485627,7.198853
Poke Bar,7.960361,10.094295,10.094295,12.885109,12.870993,15.141106,17.198266,14.959522,17.198266,14.959522,17.198266,14.959522,17.198266


In [22]:
# table of minimum cost diet compositions by age-sex groups and restaurants
diet_tbl
# it seems there is a feasible solution for each group-restaurant combination

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,"[1.6535947706889553, 1.3852752287717585, 0.709...","[2.1982570758309756, 4.11231494774835, 0.34127...","[2.198256851418826, 1.78151173378974e-06, 4.11...","[3.3071895265853226, 5.56034958804642, 0.07673...","[3.3071895423056197, 4.722821936295839, 0.1153...","[3.8562091513207064, 4.790331164253986, 0.0718...","[4.925740434841197, 0.20758255836004372, 4.847...","[3.849673202078504, 3.9686114560475874, 0.1365...","[4.947712189157788, 1.9233930755712387e-06, 3....","[3.849673201126317, 3.968611957532809, 0.13658...","[4.94771241775595, 3.8465795947305943, 0.17998...","[3.8496732024200333, 4.427298237925396, 0.0987...","[4.947712418235566, 3.8465794220684857, 0.1799..."
Ttoust,"[0.882352925603457, 1.5314822731509792, 0.2748...","[1.1764705868476217, 2.551020416219487, 0.4147...","[1.1764705880371733, 2.5510204034365738, 0.414...","[1.7647058822237875, 4.591836734301037, 0.4271...","[1.7647058820567403, 4.591836733460813, 0.4222...","[2.3529411759875267, 6.632653048006388, 0.1522...","[2.352941175766801, 0.038373509910166095, 7.42...","[0.899093957300926, 0.2563041354425244, 6.1623...","[0.7481425468840616, 0.515229448839125, 6.1869...","[0.8990940417231688, 0.2563041491157663, 6.162...","[0.7481426971027444, 0.5152294766063318, 6.186...","[2.3529411763500763, 0.05379011503111703, 7.34...","[0.7481427032552965, 0.5152294773758332, 6.186..."
IB,"[0.12140266060050481, 0.8973065484928846, 2.49...","[0.7381884201213375, 0.36265548456572066, 0.60...","[0.7381884087079148, 0.3626555521447199, 0.607...","[0.1818589658506186, 0.9773167656716606, 0.555...","[0.18185436510852976, 0.975725026242236, 0.551...","[1.2233763054815683, 1.57576435130077e-06, 2.1...","[0.5000167883018362, 0.025090358481825042, 1.0...","[1.5156130817054203, 3.8785864811010535, 0.173...","[0.5156678917103827, 0.31452057376256354, 1.20...","[1.5156130812067994, 3.8785864897315094, 0.173...","[0.41682131618836893, 0.22160690879118547, 1.0...","[0.6185488197692462, 0.7183637701925146, 0.656...","[0.41682140001715734, 0.2216078528273798, 1.01..."
Poke Bar,"[0.2786377700974017, 1.04206165823365, 4.13955...","[0.3715170277047409, 0.923279850987226, 6.4896...","[0.37151702781763934, 0.9232798518582935, 6.48...","[0.6315231559302386, 0.8115626100037986, 1.611...","[0.593710832494077, 0.7905590873998726, 1.4035...","[1.8430382611738536, 0.8342380454859687, 7.086...","[0.6937842576074815, 0.48377537465033155, 12.9...","[1.8802723382680828, 0.8456054989862727, 6.898...","[0.6937842973729921, 0.4837754199649512, 12.93...","[1.8802723187799568, 0.8456055203133529, 6.898...","[0.6937844327917748, 0.4837758079467785, 2.984...","[1.880272337983991, 0.8456054990230503, 6.8981...","[0.693784262838222, 0.4837754128303332, 12.933..."


In [69]:
# table of minimum cost diet nutrients by age-sex groups and restaurants
# saved for reference
nurt_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,"[10459.999999953641, 129.50199755271453, 34.72...","[10459.998997694707, 148.02041630488833, 43.72...","[10459.999998331097, 148.02040380939283, 43.72...","[11715.199999374792, 180.38533590544185, 51.19...","[12551.999567923745, 187.97605221334132, 51.43...","[12970.397770809583, 194.18686659278725, 52.54...","[12970.399278348923, 199.2187694663418, 51.222...","[12970.39999967084, 178.4094962975711, 50.7767...","[12970.39711703268, 182.86938986133043, 48.402...","[12970.399992092489, 178.40949624925975, 50.77...","[12970.399905738706, 182.86941554972967, 48.40...","[12970.399977480749, 188.83539022439288, 51.85...","[12970.399999003146, 182.8694152338798, 48.402..."
Ttoust,"[10459.989587448796, 70.19425615514349, 74.710...","[10430.963777238232, 84.7046757194536, 86.2084...","[10398.127026905906, 84.70468178274984, 86.208...","[11715.163823075573, 90.1892943443678, 80.0918...","[12219.607082796967, 90.18907381244074, 80.091...","[12969.555413263646, 81.73563609243551, 58.388...","[12970.384808957173, 74.46786626491082, 48.215...","[11200.195213756182, 72.49517284703869, 45.596...","[10336.455653704028, 69.99744676005318, 41.843...","[10500.54420951185, 72.49513778200725, 45.5962...","[10336.369931795512, 69.99744673196055, 41.843...","[6695.011472604339, 73.8475591582878, 48.68701...","[10336.348179273025, 69.9974468330487, 41.8437..."
IB,"[9968.188323098531, 41.29490178922483, 64.8405...","[10460.000017905637, 66.76277142336572, 65.648...","[10460.00001234891, 66.7627714254226, 65.64812...","[11715.200024350213, 65.11118670603817, 67.702...","[12114.984166597984, 64.59101427584942, 71.003...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[12088.26928313422, 73.4916263015994, 81.32894...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[12048.188298540093, 70.45991751767411, 78.935...","[12970.400090200104, 86.94985921836529, 91.519...","[12459.752818991665, 75.22744400420413, 83.184...","[12091.639268871784, 69.92913387913053, 68.305...","[12370.143725902937, 75.22744532124652, 83.184..."
Poke Bar,"[10459.999725621048, 37.57627697924011, 35.918...","[10459.998385734929, 45.20913935281094, 44.761...","[10459.999981149922, 45.20913901922931, 44.761...","[11715.199994105724, 76.70146711928513, 55.552...","[12551.999974043054, 76.73095749897972, 55.530...","[12970.395162712399, 106.00749509753444, 54.02...","[12970.40009691195, 107.59360165325603, 72.298...","[12970.398766192067, 98.28500542060023, 51.920...","[12970.400041370156, 107.5936013105872, 72.298...","[12970.399163381924, 98.28500528395313, 51.920...","[12970.400002118524, 107.59360090764085, 72.29...","[12970.39999791659, 98.28500493367888, 51.9203...","[12970.400133226072, 107.593601780598, 72.2983..."


In [24]:
# table of minimum cost diet constraining nutrients by age-sex groups and restaurants
# saved for reference
excess_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,"[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot..."
Ttoust,"[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Niacin, Potassium, K, Vitamin B-...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Energy, Calcium, Ca, Potassium, K, Vitamin B-...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita..."
IB,"[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Iron, Fe, Sodium, Na]","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Iron, Fe, Sodium, Na, Energy]","[Carbohydrate, by difference, Magnesium, Mg, T...","[Calcium, Ca, Iron, Fe, Sodium, Na, Energy]","[Carbohydrate, by difference, Magnesium, Mg, T...","[Calcium, Ca, Carbohydrate, by difference, Thi...","[Carbohydrate, by difference, Magnesium, Mg, T..."
Poke Bar,"[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Phosphorus, P, P...","[Carbohydrate, by difference, Vitamin E (alpha...","[Carbohydrate, by difference, Vitamin E (alpha...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Vitamin E (alpha...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Vitamin B-12, Vi..."


## Sensitivity of solution

### Effects of Price Changes on Subsistence Diet Cost

In [72]:
# examine effects of price change on subsistence diet cost for T-toust, the cheapest option
# for men and women aged 19-30
import cufflinks as cf
cf.go_offline()

group_pc = ['F 19-30', 'M 19-30']

for group in group_pc: 
    scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

    cost0 = solve_subsistence_problem(ing_res[1],ing_prices[1],dri_min[group],dri_max[group],tol=tol).fun

    Price_response={}
    for s in scale:
        cost = {}
        for i,p in enumerate(ing_prices[1]):
            my_p = ing_prices[1].copy()
            my_p[i] = p*s
            result = solve_subsistence_problem(ing_res[1],my_p,dri_min[group],dri_max[group],tol=tol)
            cost[ing_prices[1].index[i]] = np.log(result.fun/cost0)
        Price_response[np.log(s)] = cost

    Price_response = pd.DataFrame(Price_response).T
    Price_response.iplot(xTitle='change in log price',yTitle='change in log cost', 
                         title ='Effects of Price Changes on Subsistence Diet Cost for '+ group + ' (T-Toust)')

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!

/tmp/ipykernel_30/2046845037.py:91: UserWarning:

The solution does not satisfy the constraints within the required tolerance of 3.16E-04, yet no errors were raised and there is no certificate of infeasibility or unboundedness. Check whether the slack and constraint residuals are acceptable; if not, consider enabling presolve, adjusting the tolerance option(s), and/or using a different method. Please consider submitting a bug report.



### Effects of Price Changes on Subsistence Diet Composition

In [75]:
# examine effects of price change on subsistence diet cost for T-toust, the cheapest option
# for men and women aged 19-30
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Egg'

group_pc = ['F 19-30', 'M 19-30']

for group in group_pc: 

    scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

    cost0 = solve_subsistence_problem(ing_res[1],ing_prices[1],dri_min[group],dri_max[group],tol=tol).fun

    my_p = ing_prices[1].copy()
    diet = {}
    for s in scale:
        my_p[ReferenceGood] = ing_prices[1][ReferenceGood]*s
        result = solve_subsistence_problem(ing_res[1],my_p,dri_min[group],dri_max[group],tol=tol)
        diet[my_p[ReferenceGood]] = result.diet

    Diet_response = pd.DataFrame(diet).T
    Diet_response.index.name = '%s Price' % ReferenceGood

    Diet_response.reset_index(inplace=True)

    # Get rid of units for index (cufflinks chokes)
    # Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

    Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

    # Just look at goods consumed in quantities greater than error tolerance
    Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms',
                                                          title='Effects of Price Changes of Eggs on Subsistence Diet Composition for '+group + ' (T-Toust)')

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



/tmp/ipykernel_30/2046845037.py:91: UserWarning:

The solution does not satisfy the constraints within the required tolerance of 3.16E-04, yet no errors were raised and there is no certificate of infeasibility or unboundedness. Check whether the slack and constraint residuals are acceptable; if not, consider enabling presolve, adjusting the tolerance option(s), and/or using a different method. Please consider submitting a bug report.



## Total Cost for Population of Interest

Our population of interest is all UC berkeley students and we assume they are males and females from 19-30. Based on data from UC Berkeley Office of Planning and Analysis(https://opa.berkeley.edu/campus-data/uc-berkeley-quick-facts), there are 23,974 self-reported female students and 20,642 self-reported male students enrolled in Berkeley for Fall 2022. 

In [90]:
#Cost per capita for female and male student
cost_f_1930 = cost_tbl.loc[:,'F 19-30'].min()
cost_m_1930 = cost_tbl.loc[:,'M 19-30'].min()

num_f = 23974
num_m = 20642

#Calculation of total cost for all students
total_cost = cost_f_1930*num_f + cost_m_1930*num_m
total_cost

240099.09519716227

### Total Food Required for Population of Interest

Still, we consider to feed the population by Ttoust, our cheapest option. 

In [83]:
# retrieve saved diet compositions for male and female 19-30 
diet_m = diet_tbl.loc['Ttoust', 'M 19-30']
diet_m_name = diet_tbl_name.loc['Ttoust', 'M 19-30']
diet_f = diet_tbl.loc['Ttoust', 'F 19-30']
diet_f_name = diet_tbl_name.loc['Ttoust', 'F 19-30']

In [91]:
# list: Egg, Purple Cabbage, Spinach, Pork, Flour
# Total
egg = diet_m[0]*num_m + diet_f[0]*num_f
pc = diet_m[1]*num_m + diet_f[1]*num_f
spinach = diet_m[2]*num_m + diet_f[2]*num_f
pork = diet_m[3]*num_m + diet_f[4]*num_f
flour = diet_m[4]*num_m + diet_f[5]*num_f

In [92]:
print('We need', egg/10, 'kg eggs to feed all UC Berkeley students for one day')
print('We need', pc/10, 'kg purple cabbage to feed all UC Berkeley students for one day')
print('We need', spinach/10, 'kg spinach to feed all UC Berkeley students for one day')
print('We need', pork/10, 'kg pork to feed all UC Berkeley students for one day')
print('We need', flour/10, 'kg of flour to feed all UC Berkeley students for one day')

We need 3699.7992186362108 kg eggs to feed all UC Berkeley students for one day
We need 1677.9997903167175 kg purple cabbage to feed all UC Berkeley students for one day
We need 27544.590015871938 kg spinach to feed all UC Berkeley students for one day
We need 10349.319521782842 kg pork to feed all UC Berkeley students for one day
We need 9622.563967039674 kg of flour to feed all UC Berkeley students for one day
